# Exploratory analysis using Xorbits over NYC taxi dataset
Xorbits is a powerful tool for exploring and analyzing large datasets. One of the classic
datasets for demonstrating the capabilities of Xorbits is the NYC taxi dataset, which
contains records of taxi rides in the city from 2009 to 2022. In this blog, we will
explore how to use Xorbits to do some initial exploration of the NYC taxi dataset and get
a sense of what kind of insights we might be able to gain from the data.

## Software versions
- Xorbits==0.1.0
- plotly==5.11.0

In [ ]:
# Install dependencies
%pip install xorbits==0.1.2 plotly==5.11.0 pyarrow

## Datasets
- [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
- [NYC Taxi Zones](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc)

Download the taxi zone geojson:

In [ ]:
%%bash
wget https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc\?method\=export\&format\=GeoJSON -O taxi_zones.geojson

You can start with yellow taxi trip records of a month:

In [ ]:
%%bash
wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-12.parquet

You can also use records of a year:

In [ ]:
%%bash
for i in {1..12}
do
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-$(printf "%02d" $i).parquet
done

## Initialization
The first step is to initialize Xorbits:

In [ ]:
import xorbits

# Initialize Xorbits in the local environment.
xorbits.init()

## Data loading
The second step is to load the data into an Xorbits DataFrame. This can be done using the
`read_parquet()` function, which allows us to specify the location of the parquet file and
any additional options we want to use while reading the data.


In the case of the NYC taxi dataset, here is an example of how we could do this using
Xorbits:

In [ ]:
import datetime
import xorbits.pandas as pd

trips = pd.read_parquet(
    'yellow_tripdata_2021',
    columns=[
        'tpep_pickup_datetime',
        'tpep_dropoff_datetime',
        'trip_distance',
        'PULocationID',
        'DOLocationID',
    ])
# Remove outliers.
trips = trips[(trips['tpep_pickup_datetime'] >= datetime.datetime(2021, 1, 1)) & (trips['tpep_pickup_datetime'] <= datetime.datetime(2021, 12, 31))]
trips

In [ ]:
taxi_zones = pd.read_csv('taxi+_zone_lookup.csv', usecols=['LocationID', 'Zone'])
taxi_zones.set_index(['LocationID'], inplace=True)
taxi_zones

In [ ]:
trips.merge(taxi_zones, left_on='PULocationID', right_index=True)

In [ ]:
import json
with open('taxi_zones.geojson') as fd:
    geojson = json.load(fd)

Once we have the data loaded into a DataFrame, we might want to get a sense of the
overall structure of the data by looking at the number of rows and columns, the data types
of each column, and the first few rows of the data. We can do this using the shape, dtypes,
and head() attributes, respectively:

In [ ]:
trips.shape

In [ ]:
trips.dtypes

In [ ]:
trips.head()

## Temporal Analysis
One way to analyze the NYC taxi dataset is to look at how the number of rides varies over
time. A particularly intriguing analysis involves finding out the number of rides per hour
of each week day. We can create two additional columns in the DataFrame representing the
pickup hour and day of the week. We can then group the data by day of the week and hour
using the `groupby` method and calculate the number of rides for each group.

In [ ]:
trips['PU_dayofweek'] = trips['tpep_pickup_datetime'].dt.dayofweek
trips['PU_hour'] = trips['tpep_pickup_datetime'].dt.hour
gb_time = trips.groupby(by=['PU_dayofweek', 'PU_hour'], as_index=False).agg(count=('PU_dayofweek', 'count'))
gb_time

We can then use a library like plotly to visualize the time series data. The graph below displays
the number of rides per hour. From the graph, it is apparent that people are more inclined to
travel in the afternoon. Additionally, on weekends, people generally tend to stay out late.

In [ ]:
import plotly.express as px

b = px.bar(
    gb_time.to_pandas(),
    x='PU_hour',
    y='count',
    color='PU_dayofweek',
    color_continuous_scale='sunset_r',
)
b.show()

The graph below displays the number of rides for each day of the week.

In [ ]:
b = px.bar(
    gb_time.to_pandas(),
    x='PU_dayofweek',
    y='count',
    color='PU_hour',
    color_continuous_scale='sunset_r',
)
b.show()

## Spatial Analysis
Another way to analyze the NYC taxi dataset is to look at patterns in the spatial distribution of
rides. We can use the `groupby` method to group the data by pick-up and drop-off location ID and
compute the count of rides for each group:

In [ ]:
gb_pu_location = trips.groupby(['PULocationID'], as_index=False).agg(count=('PULocationID', 'count'))
gb_pu_location

Then visualize the spatial distribution of pick-up locations:

In [ ]:
m = px.choropleth(
    gb_pu_location.to_pandas(),
    geojson=geojson,
    locations='PULocationID',
    featureidkey='properties.location_id',
    color='count',
    color_continuous_scale='sunset_r',
    labels={'count':'trips count'}
)
m.update_geos(fitbounds='locations', visible=False)
m.show()

We can also group the data by drop-off location ID:

In [ ]:
gb_do_location = trips.groupby(['DOLocationID'], as_index=False).agg(count=('DOLocationID', 'count'))
gb_do_location

Then visualize the spatial distribution of drop-off locations:

In [ ]:
m = px.choropleth(
    gb_do_location.to_pandas(),
    geojson=geojson,
    locations='DOLocationID',
    featureidkey='properties.location_id',
    color='count',
    color_continuous_scale='sunset_r',
    labels={'count':'trips count'}
)
m.update_geos(fitbounds='locations', visible=False)
m.show()

Another area that we can explore is the traffic between taxi zones:

In [ ]:
gb_pu_do_location = trips.groupby(['PULocationID', 'DOLocationID'], as_index=False).agg(count=('PULocationID', 'count'))

# Add zone names.
gb_pu_do_location = gb_pu_do_location.merge(taxi_zones, left_on='PULocationID', right_index=True)
gb_pu_do_location.rename(columns={'Zone': 'PUZone'}, inplace=True)
gb_pu_do_location = gb_pu_do_location.merge(taxi_zones, left_on='DOLocationID', right_index=True)
gb_pu_do_location.rename(columns={'Zone': 'DOZone'}, inplace=True)

gb_pu_do_location.sort_values(['count'], inplace=True, ascending=False)
gb_pu_do_location